In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [2]:
df_model = pd.read_pickle('df_model.p')

In [3]:
df_model.head()

,count_words,count_mentions,count_hashtags,count_capital_words,count_excl_quest_marks,count_urls,count_emojis,sentiment,origin_text,likes,retweets,clean_text
557219,20,1,1,0,1,1,0,neutral,5 ways Internet of things will create new opportunities for Indian businesses!Read here http://t.co/NiroT9Xa9I #DigitalUniverseIN @emcindia,0,0,way internet thing creat new opportun indian busi read digitaluniversein
5084791,21,1,4,0,0,2,0,neutral,"As #bigdata, #cloud, and #IoT usage increases, so do #security concerns https://t.co/lXwuN7CP6s via @ITProPortal https://t.co/8KPv2ZbcuY",0,0,bigdata cloud iot usag increas secur concern
4330044,15,0,1,0,0,1,0,neutral,Hewlett Packard Enterprise Co Launches an Internet of Things Platform https://t.co/qf8qKwAhFs #iot,1,0,hewlett packard enterpris co launch internet thing platform iot
4846444,17,0,0,0,0,1,0,neutral,"The Internet of Things is about to disrupt the digital economy, report says https://t.co/RFWzNYvNY1",0,1,internet thing disrupt digit economi report say
1338071,14,0,2,0,0,1,0,neutral,How the #InternetOfThings is changing the World around Us http://t.co/iPz10oo7hc … #IoT,0,0,internetofth chang world around us iot


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [5]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [16]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions']
    

    features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                            , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [17]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.25, 0.5, 0.75)
}

In [18]:
mnb = MultinomialNB()

In [19]:
countvect = CountVectorizer()

In [20]:
# MultinomialNB
best_mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  2.0min finished


done in 126.920s

Best CV score: 0.760
Best parameters set:
	clf__alpha: 0.75
	features__pipe__vect__max_df: 0.5
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.773


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.81      0.78      0.79      1333
     neutral       0.78      0.72      0.75      1884
    positive       0.75      0.83      0.79      1730

   micro avg       0.77      0.77      0.77      4947
   macro avg       0.78      0.78      0.77      4947
weighted avg       0.77      0.77      0.77      4947

all results
0.707313 (0.003078) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.753695 (0.002086) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 2)}
0.710166 (0.002

In [11]:
tfidfvect = TfidfVectorizer()

In [12]:
best_mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.8min finished


done in 111.014s

Best CV score: 0.747
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.5
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.759


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.81      0.75      0.78      1333
     neutral       0.74      0.72      0.73      1884
    positive       0.74      0.81      0.77      1730

   micro avg       0.76      0.76      0.76      4947
   macro avg       0.76      0.76      0.76      4947
weighted avg       0.76      0.76      0.76      4947

all results
[0.68480751 0.7351871  0.68999596 0.7451597  0.69132114 0.73875837
 0.69619514 0.7470464  0.69188266 0.73860114 0.69648713 0.74677687
 0.68536903 0.71670186 0.69208481 0.74019586 0.69257895 0.72108171
 0.69846368 0.74329545 0.69264633 0.72058757 0.69880059 0.74313822
 0.68083195 0.70127128 0.69161314 0.73235704 0.688536   0.70650465
 0.6979695

In [13]:
#max_df: 0.50 or maximum document frequency of 50%.
#min_df: 1 or the words need to appear in at least 1 tweets
#ngram_range: (1, 2), both single words as bi-grams are used
#clf__alpha: 0.75
textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.5, min_df=1, ngram_range=(1,2)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', MultinomialNB(alpha=0.75))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)

In [14]:
df_model_pos = pd.read_pickle('df_model_pos.p')
best_model.predict(df_model_pos).tolist()

['positive', 'positive', 'positive']

In [15]:
df_model_neg = pd.read_pickle('df_model_neg.p')
best_model.predict(df_model_neg).tolist()

['negative', 'negative', 'negative', 'neutral']